# Trabalho de Visão Computacional

## Importação das Bibliotecas:

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import glob as gb
import os
from importlib import reload
import sys
from tqdm import tqdm
import pandas as pd

from models.alexNet import MY_AlexNet
from models.vgg16 import MY_VGG16
from utils.utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


--------------------------------------

## Configuração dos Datasets:

* CIFAR-10:

In [2]:
training_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
training_data_C10, test_data_C10

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ../data/datasets/
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ../data/datasets/
     Split: Test
     StandardTransform
 Transform: ToTensor())

* Building-vs-forests:

In [10]:
from zipfile import ZipFile

with ZipFile("../data/datasets/buildings-vs-forests/test_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")
    
with ZipFile("../data/datasets/buildings-vs-forests/traning_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")

In [ ]:
import cv2
import os
import glob as gb
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.preprocessing import LabelEncoder

trian_path = '../data/datasets/buildings-vs-forests/traning_set'
test_path = '../data/datasets/buildings-vs-forests/test_set'

#Resize the images
size_images = 150

x_train = []
y_train = []

for folder in os.listdir(trian_path):
    files = gb.glob(pathname = str(trian_path + '//' + folder + '/*jpg'))
    for file in files:
        imge = cv2.imread(file)
        #normalized images after read it
        imge = imge/255.0
        #resize images with (150,150)
        imge_arr = cv2.resize(imge ,(size_images, size_images))
        x_train.append(list(imge_arr))
        y_train.append(folder)

print(f'we have {len(x_train)} in x_train')
print(f'we have {len(y_train)} in y_train')

x_test = []
y_test = []

for folder in os.listdir(test_path):
    files = gb.glob(pathname = str(test_path + '//' + folder + '/*jpg'))
    for file in files:
        imge = cv2.imread(file)
        imge = imge/255.0
        imge_arr = cv2.resize(imge ,(size_images, size_images))
        x_test.append(list(imge_arr))
        y_test.append(folder)

print(f'we have {len(x_test)} in x_test')
print(f'we have {len(y_test)} in y_test')

---------------------

## Configuração dos Modelos:

In [4]:
batch_size = 64

# Define os dataloaders
train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

# Mostra as dimensões de cada item no iterável dataloader
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [47]:
del sys.modules['models.alexNet']
del sys.modules['models.vgg16']

from models.alexNet import MY_AlexNet
from models.vgg16 import MY_VGG16

model_1 = MY_VGG16(10)
model_2 = MY_AlexNet(10)

In [85]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_2.parameters(), lr=1e-3)

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model_2, loss_fn, optimizer, device)
    test(test_dataloader, model_2, loss_fn, device)
print("Done!")

-----------------------

## Treinamento e Avaliação - CIFAR10:

### AlexNet:

* Variando Epócas:

In [ ]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors})
df

* Variando Batch Size:

In [ ]:
batch_sizes = [32, 64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in batch_sizes:
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors})

* Variando Taxa de Aprendizado:

In [ ]:
learnings = [1e-2, 1e-3, 1e-4, 1e-5]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in batch_sizes:
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors})

* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 32
epochs = 50
learning = 1e-2

for i in range(3):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors})

### VGG16:

* Variando Épocas:

In [ ]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors})
df

* Variando Batch Size:

In [ ]:
batch_sizes = [32, 64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in batch_sizes:
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors})

* Variando Taxa de Aprendizado:

In [ ]:
learnings = [1e-2, 1e-3, 1e-4, 1e-5]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in batch_sizes:
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors})

* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

accuracy = 0
error = 999

batch_size = 32
epochs = 50
learning = 1e-2

for i in range(3):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors})

------------------------

## Treinamento e Avaliação - Building-vs-Forest: